In [30]:
import numpy as np
np.set_printoptions(precision=4)
%precision 4

u'%.4f'

<img src="Figuras/fig_025.png" alt="conf. dos cabos" width="600"/>

Three concentric neutral cables are buried in a trench with spacings as shown in Figure. 

The cables are 15 kV, 250,000 CM stranded all-aluminum with 13 strands, coated copper wires (1/3 neutral).

The outside diameter of the cable over the neutral strands is 1.29 inches (Appendix B).

Determine the phase impedance matrix and the sequence impedance matrix.

<img src="Figuras/fig_024.png" alt="conf. dos cabos" width="600"/>

<img src="Figuras/fig_023.png" alt="conf. dos cabos" width="600"/>

## Dados fornecidos pelo problema

In [31]:
dnm = 6.0 / 12.0
dod = 1.29
k = 13

## Dados dos Condutores

In [32]:
gmr = 0.0171
df = 0.567
rf = 0.41

In [33]:
gmrs = 0.0028
rs = 14.8722
ds = 0.0641

$$
R = \frac{d_{od} - d_s}{2 \times 12}
$$

In [34]:
R = (dod - ds) / (2 * 12.0)
R

0.0511

$$
GMR_{cn} = \sqrt[k]{GMR_s \cdot k \cdot R^{k-1}}
$$

In [35]:
gmrcn = (0.00208 * 13.0 * 0.0511**(13-1))**(1.0/13.0)
gmrcn

0.0487

In [36]:
rcn = rs / k
rcn

1.1440

## Montagem da matriz de distâncias

Considerando:

- Condutor 1: condutor fase 1;
- Condutor 2: condutor fase 2;
- Condutor 3: condutor fase 3;
- Condutor 4: neutro do condutor 1;
- Condutor 5: neutro do condutor 2;
- Condutor 6: neutro do condutor 3;

Distância entre condutores fase adjacentes e entre seus neutros concêntricos

In [37]:
d12 = d21 = d45 = d54 = 6.0 / 12.0

In [38]:
d23 = d32 = d56 = d65 = 6.0 / 12.0

Distância entre condutores fase que estão nos extremos e entre seus neutros concêntricos

In [39]:
d31 = d13 = d64 = d46 = 12.0 / 12.0

Distância entre os condutores e seus próprios neutros concêntricos

In [40]:
d14 = d25 = d36 = R

In [41]:
d41 = d52 = d63 = R

Uma vez que o raio dos condutores com neutro concêntricos é bem menor que a distância entre os condutores em sí, não iremos utilizar a equação:

$$
D_{ij} = \sqrt[k]{D_{nm}^k - R^{k}}
$$

Iremos utilizar apenas a distância entre os condutores concêntricos centro-a-centro, ou seja:

$$
D_{ij} = D_{nm}
$$

In [42]:
d15 = d51 = dnm

In [43]:
d24 = d42 = dnm

In [44]:
d26 = d62 = dnm

In [45]:
d35 = d53 = dnm

In [46]:
d61 = d16 = 2.0 * dnm

In [47]:
d34 = d43 = 2.0 * dnm

In [48]:
D = np.array([[gmr, d12, d13, d14, d15, d16],
              [d21, gmr, d23, d24, d25, d26],
              [d31, d32, gmr, d34, d35, d36],
              [d41, d42, d43, gmrcn, d45, d46],
              [d51, d52, d53, d54, gmrcn, d56],
              [d61, d62, d63, d64, d65, gmrcn]])
print D

[[ 0.0171  0.5     1.      0.0511  0.5     1.    ]
 [ 0.5     0.0171  0.5     0.5     0.0511  0.5   ]
 [ 1.      0.5     0.0171  1.      0.5     0.0511]
 [ 0.0511  0.5     1.      0.0487  0.5     1.    ]
 [ 0.5     0.0511  0.5     0.5     0.0487  0.5   ]
 [ 1.      0.5     0.0511  1.      0.5     0.0487]]


## Cálculo da matriz de impedâncias primitiva

Para o cálculo das impedâncias primitivas utilizamos as equações de Carson modificadas:

\begin{eqnarray}
        \hat z_{ii} & = & r_i + 0,0953 + j \cdot 0,12134 \cdot \left ( ln \frac{1}{GMR_i} + 7,93402 \right) ~~ \Omega/milha \nonumber\\
        \hat z_{ij} & = & 0,0953 + j \cdot 0,12134 \cdot \left ( ln \frac{1}{D_{ij}} + 7,93402 \right) ~~ \Omega/milha \nonumber
\end{eqnarray}

In [49]:
%precision 4

u'%.4f'

In [51]:
zabc = list()
for i in range(6):
    zl = list()
    for j in range(6):
        if i != j:
            zf = 0.0953 + 1j * (0.12134 * (np.log(1.0/D[i,j]) + 7.93402))
        else:
            if j >= 3 and i >= 3:
                zf = rcn + 0.0953 + 1j * (0.12134 * (np.log(1.0/D[i,j]) + 7.93402))
            else:
                zf = rf + 0.0953 + 1j * (0.12134 * (np.log(1.0/D[i,j]) + 7.93402))
        zl.append(zf)
    zabc.append(zl)
Z = np.array(zabc)
print Z

[[ 0.5053+1.4564j  0.0953+1.0468j  0.0953+0.9627j  0.0953+1.3236j
   0.0953+1.0468j  0.0953+0.9627j]
 [ 0.0953+1.0468j  0.5053+1.4564j  0.0953+1.0468j  0.0953+1.0468j
   0.0953+1.3236j  0.0953+1.0468j]
 [ 0.0953+0.9627j  0.0953+1.0468j  0.5053+1.4564j  0.0953+0.9627j
   0.0953+1.0468j  0.0953+1.3236j]
 [ 0.0953+1.3236j  0.0953+1.0468j  0.0953+0.9627j  1.2393+1.3295j
   0.0953+1.0468j  0.0953+0.9627j]
 [ 0.0953+1.0468j  0.0953+1.3236j  0.0953+1.0468j  0.0953+1.0468j
   1.2393+1.3295j  0.0953+1.0468j]
 [ 0.0953+0.9627j  0.0953+1.0468j  0.0953+1.3236j  0.0953+0.9627j
   0.0953+1.0468j  1.2393+1.3295j]]


## Particionamento da matriz de impedâncias primitiva para redução de Kron

In [52]:
zij = Z[0:3, 0:3]
zij = np.reshape(zij, (3, 3))
print zij

[[ 0.5053+1.4564j  0.0953+1.0468j  0.0953+0.9627j]
 [ 0.0953+1.0468j  0.5053+1.4564j  0.0953+1.0468j]
 [ 0.0953+0.9627j  0.0953+1.0468j  0.5053+1.4564j]]


In [53]:
zin = Z[0:3, 3:6]
zin = np.reshape(zin, (3, 3))
print zin

[[ 0.0953+1.3236j  0.0953+1.0468j  0.0953+0.9627j]
 [ 0.0953+1.0468j  0.0953+1.3236j  0.0953+1.0468j]
 [ 0.0953+0.9627j  0.0953+1.0468j  0.0953+1.3236j]]


In [54]:
znj = Z[3:6, 0:3]
znj = np.reshape(znj, (3, 3))
print znj

[[ 0.0953+1.3236j  0.0953+1.0468j  0.0953+0.9627j]
 [ 0.0953+1.0468j  0.0953+1.3236j  0.0953+1.0468j]
 [ 0.0953+0.9627j  0.0953+1.0468j  0.0953+1.3236j]]


In [55]:
znn = Z[3:6,3:6]
znn = np.reshape(znn, (3, 3))
print znn

[[ 1.2393+1.3295j  0.0953+1.0468j  0.0953+0.9627j]
 [ 0.0953+1.0468j  1.2393+1.3295j  0.0953+1.0468j]
 [ 0.0953+0.9627j  0.0953+1.0468j  1.2393+1.3295j]]


## Aplicação da redução de Kron

A equação que representa a redução de Kron é dada por:

\begin{equation}
        \mathbf{\hat z_{abc}} = (\mathbf{\hat z_{ij}} - \mathbf{\hat z_{in}} \cdot \mathbf{\hat z^{-1}_{nn}} \cdot \mathbf{\hat z_{nj}} )
\end{equation}

In [56]:
Zabc = zij - np.dot(np.dot(zin, np.linalg.inv(znn)) ,znj)
Zabc

array([[ 0.7982+0.4463j,  0.3192+0.0328j,  0.2849-0.0143j],
       [ 0.3192+0.0328j,  0.7891+0.4041j,  0.3192+0.0328j],
       [ 0.2849-0.0143j,  0.3192+0.0328j,  0.7982+0.4463j]])

## Cálculo da matriz de impedâncias de sequência

\begin{equation}
    \mathbf{Z_{012}} = \mathbf{A}^{-1} \cdot \mathbf{Z_{abc}} \cdot \mathbf{A} = 
    \left[
    \begin{array}{ccc}
             z_{00} & z_{01} & z_{02} \\
             z_{10} & z_{11} & z_{12} \\
             z_{20} & z_{21} & z_{22}
    \end{array} 
    \right]  
\end{equation}

Em que $A$:

\begin{equation}
\mathbf{A} = 
    \left[ 
    \begin{array}{ccc}
           1 & 1 & 1\\
           1 & a^2 & a\\
           1 & a & a^2
    \end{array} 
    \right]
\end{equation}

Com:

$$
a = 1 \angle 120^{o}
$$

In [57]:
a = -0.5 + 1j * np.sqrt(3.0) / 2.0

In [58]:
A = np.array([[1.0, 1.0, 1.0], [1.0, a**2, a], [1.0, a, a**2]])
print A

[[ 1.0+0.j     1.0+0.j     1.0+0.j   ]
 [ 1.0+0.j    -0.5-0.866j -0.5+0.866j]
 [ 1.0+0.j    -0.5+0.866j -0.5-0.866j]]


In [59]:
Z012 = np.dot(np.linalg.inv(A), np.dot(Zabc, A))
print Z012

[[ 1.4107 +4.6646e-01j -0.0028 -8.0950e-03j -0.0056 +6.4517e-03j]
 [-0.0056 +6.4517e-03j  0.4874 +4.1513e-01j -0.0264 +4.5123e-02j]
 [-0.0028 -8.0950e-03j  0.0523 +3.3724e-04j  0.4874 +4.1513e-01j]]
